# Explainability fo physicians

## Load cached train test set

In [1]:
import pickle
import numpy as np
import pandas as pd

train_test_pickle_path = open("cache/train_test.pickle","rb")
data_dict = pickle.load(train_test_pickle_path)

train_x = data_dict['train_x']
train_y = data_dict['train_y']
test_x = data_dict['test_x']
test_y = data_dict['test_y']

## Instantiate the model
This is the model that was found to be the most effective during the Model Selection process

In [2]:
from sklearn.tree import DecisionTreeClassifier

decision_tree_classifier = DecisionTreeClassifier(max_depth=4, min_samples_leaf=0.166529153009486, min_samples_split=3)

In [3]:
clf = decision_tree_classifier.fit(train_x, train_y) # Use the model selected in the previous step to predict train points

p_train = clf.predict(train_x)

p_train = p_train.reshape((p_train.shape[0],1))


In [4]:
z_train = np.hstack((train_x, p_train)) # Store instances that were predicted as MCI
z_train_mci = z_train[z_train[:,-1]==1, :]

In [5]:
idx = 2

class_names = ['Healthy', 'MCI', 'AD']

df = test_x

test_x = test_x.to_numpy()
test_y = test_y.to_numpy()


In [6]:
X = test_x[idx].reshape((1,) + test_x[idx].shape)

print("Chosen Sample:", idx)
print("Prediction made by the model:", class_names[np.argmax(clf.predict_proba(X))])
print("Prediction probabilities:", clf.predict_proba(X))
print("")

# attach the prediction made by the model to X
X = np.hstack((X, clf.predict(X).reshape((1,1))))

#Xun = test_x[idx].reshape((1,) + x_test[idx].shape) 
# dfx = pd.DataFrame.from_records(X.astype('double')) # Create dataframe with original feature values
# #dfx[23] = class_names[int(X[0, -1])]
# dfx.columns = df.columns
# dfx.transpose()

Chosen Sample: 2
Prediction made by the model: MCI
Prediction probabilities: [[0. 1. 0.]]



#### Apply Protodash

In [7]:
from aix360.algorithms.protodash import ProtodashExplainer

explainer = ProtodashExplainer()
(W, S, setValues) = explainer.explain(X, z_train_mci, m=5) # Return weights W, Prototypes S and objective function values

     pcost       dcost       gap    pres   dres
 0:  0.0000e+00 -2.0000e+04  4e+00  1e+00  1e+00
 1:  1.7611e+04 -6.5756e+06  1e+03  1e+00  1e+00
 2: -1.8164e+04 -1.4525e+07  3e+03  1e+00  1e+00
 3:  5.7917e+04 -9.9149e+08  1e+05  1e+00  1e+00
 4:  5.2309e+08 -5.3957e+12  1e+09  1e+00  1e+00
 5:  5.3082e+08 -5.4570e+12  2e+09  1e+00  1e+00
 6:  2.6599e+08 -1.0671e+13  5e+09  1e+00  1e+00
 7: -6.6650e+07 -1.0244e+14  3e+10  1e+00  1e+00
 8:  5.2092e+08 -1.5503e+15  4e+11  1e+00  1e+00
 9:  1.5446e+14 -1.9801e+22  2e+22  4e-13  2e-06
10:  1.5446e+14 -1.9801e+20  2e+20  4e-15  2e-06
11:  1.5445e+14 -1.9802e+18  2e+18  2e-16  1e-08
12:  1.5331e+14 -1.9963e+16  2e+16  2e-16  1e-10
13:  8.9938e+13 -2.9284e+14  4e+14  2e-16  2e-12
14:  1.4217e+13 -1.6676e+13  3e+13  2e-16  4e-13
15:  2.0314e+12 -2.2663e+12  4e+12  2e-16  1e-13
16:  2.8833e+11 -3.2941e+11  6e+11  2e-16  3e-14
17:  4.0161e+10 -4.8268e+10  9e+10  2e-16  3e-17
18:  5.2820e+09 -7.3698e+09  1e+10  3e-16  8e-15
19:  5.5013e+08 -1.24

/evo970/conda/ibmExp/lib/python3.7/site-packages/cvxopt/coneprog.py:2111: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'x' in initvals:
/evo970/conda/ibmExp/lib/python3.7/site-packages/cvxopt/coneprog.py:2116: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 's' in initvals:
/evo970/conda/ibmExp/lib/python3.7/site-packages/cvxopt/coneprog.py:2131: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'y' in initvals:
/evo970/conda/ibmExp/lib/python3.7/site-packages/cvxopt/coneprog.py:2136: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'z' in initvals:


## Results

In [8]:
dfs = pd.DataFrame.from_records(z_train_mci[S, 0:-1].astype('double'))
RP=[]
for i in range(S.shape[0]):
    RP.append(class_names[int(z_train_mci[S[i], -1])]) # Append class names
# dfs[23] = RP
dfs.columns = df.columns  
dfs["Weight"] = np.around(W, 5)/np.sum(np.around(W, 5)) # Calculate normalized importance weights
dfs.transpose()

,0,1,2,3,4
Fp1_delta_mean,113.476063,19219.587300,20.542651,155.388533,4474.565716
Fp1_theta_mean,0.889057,22.662003,0.705713,4.329504,13.460706
Fp1_alpha_mean,0.370064,1.131999,0.836354,356.222337,0.272787
Fp1_beta_mean,0.021636,0.140163,0.091580,1.065358,0.528827
Fp1_gamma_mean,0.002600,0.012649,0.007493,0.104261,0.066491
...,...,...,...,...,...
Pz_theta_mean,0.212950,1.088364,0.309611,1.060286,0.119045
Pz_alpha_mean,0.366207,2.009488,0.500757,345.852941,0.033510
Pz_beta_mean,0.017727,0.178735,0.056373,0.880907,0.003546
Pz_gamma_mean,0.000539,0.003187,0.001153,0.003029,0.000287


### More Choices for Feature Selection

Thus with a better feature selection on the previous step to cut of some of the features the physitian may have a reference from which to take a decition in a more accurate manner